In [ ]:
pip install flask


In [ ]:
pip install --upgrade pip setuptools


In [ ]:
pip install scipy==1.7.3


In [ ]:
pip install -r requirements.txt


In [ ]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
pip setup.py install 


In [ ]:
pip setup.py develop

In [ ]:
pip install transformers

In [ ]:

import json
import urllib
import opennre

In [ ]:

from flask import Flask, request


In [ ]:
pip install --upgrade spacy

In [ ]:
import spacy
from typing import List
from spacy.tokens import Doc, Span
from fastcoref import FCoref
#from fastcoref import LingMessCoref
# from flask import Flask, render_template, request, redirect, session, jsonify

nlp = spacy.load('en_core_web_sm')
model = FCoref()
#model = LingMessCoref(device='cuda:0')
# app = Flask(__name__)

def core_logic_part(document: Doc, coref: List[int], resolved: List[str], mention_span: Span):
    final_token = document[coref[1]]
    if final_token.tag_ in ["PRP$", "POS"]:
        resolved[coref[0]] = mention_span.text + "'s" + final_token.whitespace_
    else:
        resolved[coref[0]] = mention_span.text + final_token.whitespace_
    for i in range(coref[0] + 1, coref[1] + 1):
        resolved[i] = ""
    return resolved

def get_span_noun_indices(doc: Doc, cluster: List[List[int]]) -> List[int]:
    spans = [doc[span[0]:span[1]+1] for span in cluster]
    spans_pos = [[token.pos_ for token in span] for span in spans]
    span_noun_indices = [i for i, span_pos in enumerate(spans_pos)
        if any(pos in span_pos for pos in ['NOUN', 'PROPN'])]
    return span_noun_indices

def get_cluster_head(doc: Doc, cluster: List[List[int]], noun_indices: List[int]):
    head_idx = noun_indices[0]
    head_start, head_end = cluster[head_idx]
    head_span = doc[head_start:head_end+1]
    return head_span, [head_start, head_end]

def is_containing_other_spans(span: List[int], all_spans: List[List[int]]):
    return any([s[0] >= span[0] and s[1] <= span[1] and s != span for s in all_spans])

def improved_replace_corefs(document, clusters):
    resolved = list(tok.text_with_ws for tok in document)
    all_spans = [span for cluster in clusters for span in cluster]  # flattened list of all spans

    for cluster in clusters:
        noun_indices = get_span_noun_indices(document, cluster)

        if noun_indices:
            mention_span, mention = get_cluster_head(document, cluster, noun_indices)

            for coref in cluster:
                if coref != mention and not is_containing_other_spans(coref, all_spans):
                    core_logic_part(document, coref, resolved, mention_span)

    return "".join(resolved)

def get_fast_cluster_spans(doc, clusters):
    fast_clusters = []
    for cluster in clusters:
        new_group = []
        for tuple in cluster:
            # print(type(tuple), tuple)
            (start, end) = tuple
            # print("start, end", start, end)
            span = doc.char_span(start, end)
            # print('span', span.start, span.end)
            new_group.append([span.start, span.end-1])
        fast_clusters.append(new_group)
    return fast_clusters

def get_fastcoref_clusters(doc, text):
    preds = model.predict(texts=[text])
    fast_clusters = preds[0].get_clusters(as_strings=False)
    fast_cluster_spans = get_fast_cluster_spans(doc, fast_clusters)
    return fast_cluster_spans

# @app.route('/coreference', methods=['POST'])
def coreference(text):
    #print(request.form["test"])
    # content = request.get_json()
    # print('content', content)
    # text = content["text"]
   
    doc = nlp(text)
    #clusters = get_allennlp_clusters(text)
    clusters = get_fastcoref_clusters(doc, text)
    coref_text = improved_replace_corefs(doc, clusters)
    print('coref_text', coref_text)
    return coref_text


In [ ]:
import json
import urllib
from string import punctuation
import nltk

# Rest of your code...


ENTITY_TYPES = ["human", "person", "company", "enterprise", "business", "geographic region","date","duration", "time",
                "human settlement", "geographic entity", "territorial entity type", "organization"]

# def wikifier(text, lang="en", threshold=0.5):
#     """Function that fetches entity linking results from wikifier.com API"""
#     # Prepare the URL.
#     data = urllib.parse.urlencode([
#         ("text", text), ("lang", lang),
#         ("userKey", "tgbdmkpmkluegqfbawcwjywieevmza"),
#         ("pageRankSqThreshold", "%g" %
#          threshold), ("applyPageRankSqThreshold", "true"),
#         ("nTopDfValuesToIgnore", "100"), ("nWordsToIgnoreFromList", "100"),
#         ("wikiDataClasses", "true"), ("wikiDataClassIds", "false"),
#         ("support", "true"), ("ranges", "false"), ("minLinkFrequency", "2"),
#         ("includeCosines", "false"), ("maxMentionEntropy", "3")
#     ])
#     url = "http://www.wikifier.org/annotate-article"
#     # Call the Wikifier and read the response.
#     req = urllib.request.Request(url, data=data.encode("utf8"), method="POST")
#     with urllib.request.urlopen(req, timeout=60) as f:
#         response = f.read()
#         response = json.loads(response.decode("utf8"))
#     # Output the annotations.
#     results = list()
#     for annotation in response["annotations"]:
#         # Filter out desired entity classes
#         if ('wikiDataClasses' in annotation) and (any([el['enLabel'] in ENTITY_TYPES for el in annotation['wikiDataClasses']])):

#             # Specify entity label
#             if any([el['enLabel'] in ["human", "person"] for el in annotation['wikiDataClasses']]):
#                 label = 'Person'
#             elif any([el['enLabel'] in ["company", "enterprise", "business", "organization"] for el in annotation['wikiDataClasses']]):
#                 label = 'Organization'
#             elif any([el['enLabel'] in ["geographic region", "human settlement", "geographic entity", "territorial entity type"] for el in annotation['wikiDataClasses']]):
#                 label = 'Location'
#             elif any([el['enLabel'] in ["date", "duration"] for el in annotation['wikiDataClasses']]):
#                 label = 'Date/ Duration'
#             else:
#                 label = None

#             results.append({'title': annotation['title'], 'wikiId': annotation['wikiDataItemId'], 'label': label,
#                             'characters': [(el['chFrom'], el['chTo']) for el in annotation['support']]})
#     return results

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")
def kauling(text):
    doc = nlp(text)
    results=list()
    for ent in doc.ents:
         results.append({'Entity': ent.text,
                          'Type': ent.label_, 
                          'Category': spacy.explain(ent.label_),
                          'start_char': ent.start_char,
                            'end_char': ent.end_char,
                            'characters': [(ent.start_char, ent.end_char)]})

    return results

In [ ]:
from opennre.pretrain import get_model
import itertools
import csv
import nltk
import spacy

nlp = spacy.load("en_core_web_sm")
nltk.download('punkt')
relation_model = get_model('wiki80_cnn_softmax')
ENTITY_TYPES = ["human", "person", "company", "enterprise", "business", "geographic region",
                "human settlement", "geographic entity", "territorial entity type", "organization"]

def strip_punctuation(s):
    """Removes all punctuation from a string"""
    return ''.join(c for c in s if c not in punctuation)


def deduplicate_dict(d):
    return [dict(y) for y in set(tuple(x.items()) for x in d)]


def final(text):
   
        relation_threshold=0.5
        entities_threshold=0.8
        coref =  True
        if not text:
            return 'Missing text parameter'

        try:
            relation_threshold = float(relation_threshold)
            entities_threshold = float(entities_threshold)
        except ValueError:
            return 'Invalid value for relation or entity threshold parameter'

        if coref:
            text = coreference(text)

        print(text)

        relations_list = list()
        entities_list = list()

        for sentence in nltk.sent_tokenize(text):
            sentence = strip_punctuation(sentence)
            # entities = wikifier(sentence, threshold=entities_threshold)
            entities=kauling(sentence)
            entities_list.extend(
                [{'title': el['Entity'], 'label': el['Type'], 'Category': el['Category']} for el in entities])
            # Iterate over every permutation pair of entities
            for permutation in itertools.permutations(entities, 2):
                for source in permutation[0]['characters']:
                    for target in permutation[1]['characters']:
                        # Relationship extraction with OpenNRE
                        data = relation_model.infer(
                            {'text': sentence, 'h': {'pos': [source[0], source[1] + 1]}, 't': {'pos': [target[0], target[1] + 1]}})
                        if data[1] > relation_threshold:
                            relations_list.append(
                                {'source': permutation[0]['Entity'], 'target': permutation[1]['Entity'], 'type': data[0]})
        result = {'entities': deduplicate_dict(entities_list), 'relations': deduplicate_dict(relations_list)}
        with open('entities.csv', 'w', newline='', encoding='utf-8') as entities_file:
            entities_writer = csv.DictWriter(entities_file, fieldnames=['title', 'label', 'Category'])
            entities_writer.writeheader()
            entities_writer.writerows(entities_list)

        # Write relations to CSV
        with open('relations.csv', 'w', newline='', encoding='utf-8') as relations_file:
            relations_writer = csv.DictWriter(relations_file, fieldnames=['source', 'target', 'type'])
            relations_writer.writeheader()
            relations_writer.writerows(relations_list)
        print("Entities:")
        for entity in entities_list:
            print(f"{entity['title']}: {entity['label']}")

        # Print relations for current sentence
        print("\nRelations:")
        for relation in relations_list:
            print(f"{relation['source']} -- {relation['type']} --> {relation['target']}")
    

text=""" Napoleon Bonaparte (born Napoleone di Buonaparte;[1][b] 15 August 1769 – 5 May 1821), later known by his regnal name Napoleon I, was a French emperor and military commander who rose to prominence during the French Revolution and led successful campaigns during the Revolutionary Wars. He was the leader of the French Republic as First Consul from 1799 to 1804, then of the French Empire as Emperor of the French from 1804 until 1814, and briefly again in 1815. His political and cultural legacy endures as a celebrated and controversial leader. He initiated many enduring reforms, but has been criticized for his authoritarian rule. He is considered one of the greatest military commanders in history and his wars and campaigns are still studied at military schools worldwide. However, historians still debate the degree to which he was responsible for the Napoleonic Wars, in which between three and six million people died.[2][3]

Napoleon was born on the island of Corsica into a family descended from Italian nobility.[4][5] He was resentful of the French monarchy, and supported the French Revolution in 1789 while serving in the French army, trying to spread its ideals to his native Corsica. He rose rapidly in the ranks after saving the governing French Directory by firing on royalist insurgents. In 1796, he began a military campaign against the Austrians and their Italian allies, scoring decisive victories, and became a national hero. Two years later he led a military expedition to Egypt that served as a springboard to political power. He engineered a coup in November 1799 and became First Consul of the Republic. In 1804, to consolidate and expand his power, he crowned himself Emperor of the French.

Differences with the United Kingdom meant France faced the War of the Third Coalition by 1805. Napoleon shattered this coalition with victories in the Ulm campaign and at the Battle of Austerlitz, which led to the dissolution of the Holy Roman Empire. In 1806, the Fourth Coalition took up arms against him. Napoleon defeated Prussia at the battles of Jena and Auerstedt, marched the Grande Armée into Eastern Europe, and defeated the Russians in June 1807 at Friedland, forcing the defeated nations of the Fourth Coalition to accept the Treaties of Tilsit. Two years later, the Austrians challenged the French again during the War of the Fifth Coalition, but Napoleon solidified his grip over Europe after triumphing at the Battle of Wagram.

Hoping to extend the Continental System, his embargo against Britain, Napoleon invaded the Iberian Peninsula and declared his brother Joseph the King of Spain in 1808. The Spanish and the Portuguese revolted in the Peninsular War aided by a British army, culminating in defeat for Napoleon's marshals. Napoleon launched an invasion of Russia in the summer of 1812. The resulting campaign witnessed the catastrophic retreat of Napoleon's Grande Armée. In 1813, Prussia and Austria joined Russian forces in a Sixth Coalition against France, resulting in a large coalition army defeating Napoleon at the Battle of Leipzig. The coalition invaded France and captured Paris, forcing Napoleon to abdicate in April 1814. He was exiled to the island of Elba, between Corsica and Italy. In France, the Bourbons were restored to power.

Napoleon escaped in February 1815 and took control of France.[6] The Allies responded by forming a Seventh Coalition, which defeated Napoleon at the Battle of Waterloo in June 1815. The British exiled him to the remote island of Saint Helena in the Atlantic, where he died in 1821 at the age of 51.

Napoleon had a lasting impact on the world, bringing modernizing reforms to France and Western Europe[c] and stimulating the development of nation states. He also sold the Louisiana Territory to the United States in 1803, doubling the latter's size.[2][13] However, his mixed record on civil rights and exploitation of conquered territories adversely affect his reputation.[d]

Early life
Napoleon's family was of Italian origin. His paternal ancestors, the Buonapartes, descended from a minor Tuscan noble family that emigrated to Corsica in the 16th century and his maternal ancestors, the Ramolinos, descended from a noble family from Lombardy.[18]

Half-length portrait of a wigged middle-aged man with a well-to-do jacket. His left hand is tucked inside his waistcoat.
Napoleon's father, Carlo Buonaparte, fought for Corsican independence under Pasquale Paoli, but after their defeat he eventually became the island's representative to Louis XVI's court.
His parents Carlo Maria Buonaparte and Maria Letizia Ramolino maintained a home in Ajaccio where Napoleon was born on 15 August 1769. He had an elder brother, Joseph, and, later, six younger siblings: Lucien, Elisa, Louis, Pauline, Caroline, and Jérôme.[19] Five more siblings were stillborn or did not survive infancy.[20] Napoleon was baptized as a Catholic, under the name Napoleone di Buonaparte. In his youth, his name was also spelled as Nabulione, Nabulio, Napolionne, and Napulione.[21]

Napoleon was born one year after the Republic of Genoa ceded Corsica to France.[22][e] His father fought alongside Pasquale Paoli during the Corsican war of independence against France. After the Corsican defeat at Ponte Novu in 1769 and Paoli's exile in Britain, Carlo became friends with the French governor Charles Louis de Marbeuf, who became his patron and godfather to Napoleon.[26][27] With Mabeuf's support, Carlo was named Corsican representative to the court of Louis XVI and Napoleon obtained a royal bursary to a military academy in France.[28][29]"""
# ctext=coreference(text)
# text="Akbar was born in 1987"
final(text)